In [ ]:
import folium
import pandas as pd
import geopandas as gpd

%load_ext watermark

In [ ]:
%watermark -a "João Palmeiro" -u -d -v -p folium,pandas,geopandas

In [ ]:
URL_GEOJSON = "https://raw.githubusercontent.com/joaopalmeiro/portugal-maps/master/geojson/ccdr_portugal_continental.geojson"
URL_DATA = "https://raw.githubusercontent.com/dssg-pt/covid19pt-data/master/data.csv"

In [ ]:
df_map = gpd.read_file(URL_GEOJSON)

In [ ]:
data = pd.read_csv(URL_DATA)

In [ ]:
df_most_recent_date = data.tail(1)

In [ ]:
confirmed_col = {
    "Alentejo": df_most_recent_date["confirmados_arsalentejo"].item(),
    "Algarve": df_most_recent_date["confirmados_arsalgarve"].item(),
    "Centro": df_most_recent_date["confirmados_arscentro"].item(),
    "Norte": df_most_recent_date["confirmados_arsnorte"].item(), 
    "RLVT": df_most_recent_date["confirmados_arslvt"].item()
}

In [ ]:
df_map["confirmed"] = df_map["CCDR"].map(confirmed_col)

In [ ]:
df_map

In [ ]:
m = folium.Map(location=[39.557191, -7.8536599], zoom_start=6, tiles="cartodbpositron")

tooltip = folium.features.GeoJsonTooltip(
    fields=["CCDR", "confirmed"],
    aliases=["Regional Health Administration:", "Number of Confirmed Cases:"]
)

choropleth = folium.Choropleth(
    geo_data=df_map,
    name='choropleth',
    data=df_map[['CCDR', 'confirmed']],
    columns=['CCDR', 'confirmed'],
    key_on='feature.properties.CCDR',
    fill_color='Blues',
    fill_opacity=0.9,
    line_opacity=0.5,
    legend_name=f'Number of Confirmed Cases: {df_most_recent_date["data"].item()}'
).add_to(m)

choropleth.geojson.add_child(
    tooltip
)

folium.LayerControl('topleft').add_to(m)

m